In [1]:
import pandas as pd
import math
import numpy as np
import collections

In [2]:
medication=pd.read_csv("medication.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
medication.sort_values(by=['drugorderoffset'],inplace=True)

In [5]:
microlab=pd.read_csv("microLab.csv")
microlab.head()

,microlabid,patientunitstayid,culturetakenoffset,culturesite,organism,antibiotic,sensitivitylevel
0,549559,983524,-688,"Blood, Venipuncture",gram positive rods,NaN,NaN
1,549555,983524,-5481,Stool,no growth,NaN,NaN
2,549565,983524,324,Stool,no growth,NaN,NaN
3,549562,983524,-686,"Blood, Central Line",gram negative rods,NaN,NaN
4,549564,983524,-681,"Blood, Central Line",gram positive cocci,NaN,NaN


In [6]:
microlab.sort_values(by=['culturetakenoffset'],inplace=True)

In [7]:
blood_cultures=microlab.loc[microlab['culturesite'].str.contains('Blood')]

In [8]:
count=0
for val in medication['routeadmin']:
    if val=='IV':
        count+=1

In [10]:
medication_2=medication.loc[medication['routeadmin']=='IV']


In [13]:
antibiotics_list=['adoxa','ala-tet','alodox','amikacin','amikin','amoxicillin','clavulanate','ampicillin',
                 'augmentin','avelox','avidoxy','azactam','azithromycin','aztreonam','axetil','bactocill',
                 'bactrim','bethkis','biaxin','bicillin l-a','cayston','cefazolin','cedax','cefoxitin',
                 'ceftazidime','cefaclor','cefadroxil','cefdinir','cefditoren','cefepime','cefotetan','cefotaxime',
                 'cefpodoxime','cefprozil','ceftibuten','ceftin','cefuroxime','cefuroxime','cephalexin','chloramphenicol',
                 'cipro','ciprofloxacin','claforan','clarithromycin','cleocin','clindamycin','cubicin','dicloxacillin',
                 'doryx','doxycycline','duricef','dynacin','ery-tab','eryped','eryc','erythrocin','erythromycin','factive',
                 'flagyl','fortaz','furadantin','garamycin','gentamicin','kanamycin','keflex','ketek','levaquin',
                 'levofloxacin','lincocin','macrobid','macrodantin','maxipime','mefoxin','metronidazole','minocin',
                 'minocycline','monodox','monurol','morgidox','moxatag','moxifloxacin','myrac','nafcillin sodium',
                 'nicazel doxy 30','nitrofurantoin','noroxin','ocudox','ofloxacin','omnicef','oracea','oraxyl','oxacillin',
                 'pc pen vk','pce dispertab','panixine','pediazole','penicillin','periostat','pfizerpen','piperacillin',
                 'tazobactam','primsol','proquin','raniclor','rifadin','rifampin','rocephin','smz-tmp','septra','septra ds',
                 'septra','solodyn','spectracef','streptomycin sulfate','sulfadiazine','sulfamethoxazole','trimethoprim',
                 'sulfatrim','sulfisoxazole','suprax','synercid','tazicef','tetracycline','timentin','tobi','tobramycin',
                 'trimethoprim','unasyn','vancocin','vancomycin','vantin','vibativ','vibra-tabs','vibramycin','zinacef',
                 'zithromax','zmax','zosyn','zyvox']

med_antibiotics=[]
for row in medication_2.itertuples():
    if(any(sub in str(row.drugname).lower() for sub in antibiotics_list)):
        med_antibiotics.append(row)

In [20]:
med_nonIV_anti=[]
for row in medication.itertuples():
    if(any(sub in str(row.drugname).lower() for sub in antibiotics_list)):
        med_nonIV_anti.append(row)

In [22]:
med_nonIV_anti=pd.DataFrame(med_nonIV_anti)
med_nonIV_anti.drop(columns=['Index'],inplace=True)

In [14]:
med_antibiotics=pd.DataFrame(med_antibiotics)
med_antibiotics.drop(columns=['Index'],inplace=True)


In [18]:
len(med_antibiotics)

79754

In [16]:
t_sus_culture={}
for pid in blood_cultures['patientunitstayid'].unique():
    
    cultures=blood_cultures.loc[blood_cultures['patientunitstayid']==pid]
    cultures.sort_values(by=['culturetakenoffset'],inplace=True)
    earliest_culture=cultures['culturetakenoffset'].iloc[0]
    
    medication_pid=med_antibiotics.loc[med_antibiotics['patientunitstayid']==pid]
    medication_pid.sort_values(by=['drugorderoffset'],inplace=True)
    
    for i in range(len(medication_pid)):
        if(medication_pid['drugorderoffset'].iloc[i]>earliest_culture):
            timediff=abs(medication_pid['drugorderoffset'].iloc[i]-earliest_culture)
            
            if(timediff>(72*60)):
                break
            else:
                t_sus_culture[pid]=earliest_culture
                break

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [17]:
t_sus_culture

{}

In [24]:
for pid in microlab['patientunitstayid'].unique():
    print(med_nonIV_anti.loc[med_nonIV_anti['patientunitstayid']==pid])

Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, d

Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, d

Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
        medicationid  patientunitstayid  drugorderoffset  dr

Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, d

Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientuni

...

        medicationid  patientunitstayid  drugorderoffset  drugstartoffset  \
82957       91111673            2643294              208              232   
82959       90479032            2643294              208              952   
130844      91160023            2643294             3958             4282   

       drugivadmixture drugordercancelled        drugname  drughiclseqno  \
82957              Yes                 No  VANCOMYCIN HCL         4042.0   
82959              Yes                Yes  VANCOMYCIN HCL         4042.0   
130844             Yes                Yes  VANCOMYCIN HCL         4042.0   

        dosage routeadmin frequency loadingdose prn  drugstopoffset  gtc  
82957     2 GM         IV       ONE         NaN  No          525921   19  
82959   500 MG         IV      Q12H         NaN  No           21111   19  
130844    1 GM         IV      Q12H         NaN  No           24441   19  
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstarto

Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
        medicationid  patientunitstayid  drugorderoffset  drugstartoffset  \
113513     102251241            3074395             2017             2028   
113525     102251306            3074395             2018             2017   
123391     102252490            3074395             2954             2958   
128508     102253312            3074395             3664             3678   

       drugivadmixture drugordercancelled                     drugname  \
113513              No                 No      PIPERACILLIN/TAZOBACTAM   
113525              No                 No  VANCOMYCIN 1.25 G/250 ML NS   
123391              No                 No      PIPERACILLIN/TAZOBACTAM   
128508              No                 No    VANCOMYCIN 1 G/200 ML D5W   

        

       medicationid  patientunitstayid  drugorderoffset  drugstartoffset  \
64168      91566149            2599275               48               57   
65021      90818795            2599275               51              837   

      drugivadmixture drugordercancelled    drugname  drughiclseqno   dosage  \
64168             Yes                 No  VANCOMYCIN         4042.0  1500 MG   
65021             Yes                 No  VANCOMYCIN         4042.0   750 MG   

      routeadmin frequency loadingdose prn  drugstopoffset  gtc  
64168         IV       OTO         NaN  No              58   20  
65021         IV     Q12HR         NaN  No           10773   20  
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drug

Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, d

        medicationid  patientunitstayid  drugorderoffset  drugstartoffset  \
129209      90025602            2580880             3751             3760   
130843      91805433            2580880             3958             4480   

       drugivadmixture drugordercancelled    drugname  drughiclseqno   dosage  \
129209             Yes                 No  VANCOMYCIN         4042.0  2000 MG   
130843             Yes                 No  VANCOMYCIN         4042.0  1250 MG   

       routeadmin frequency loadingdose prn  drugstopoffset  gtc  
129209         IV       OTO         NaN  No            3761   20  
130843         IV      Q8HR         NaN  No           10181   20  
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoffset, drugivadmixture, drugordercancelled, drugname, drughiclseqno, dosage, routeadmin, frequency, loadingdose, prn, drugstopoffset, gtc]
Index: []
Empty DataFrame
Columns: [medicationid, patientunitstayid, drugorderoffset, drugstartoff